In [15]:
import os
import sys
import geopandas as gpd
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col, split, when, regexp_extract, round, regexp_replace, format_number
from pyspark.sql.types import StringType
from shapely.geometry import Point

sys.path.append('/home/jiayin/assessment-rent-airbnb/src')

from bronze.ingestion import ingest_data
from silver.cleaning import get_cleaned_df
from gold.aggregation import get_avg_rev

In [16]:
if __name__ == "__main__":
    # ingest_data()

    airbnb_cleaned = get_cleaned_df('../../data/airbnb.csv')
    rentals_cleaned = get_cleaned_df('../../data/rentals.json')
    rentals_rev = get_avg_rev(rentals_cleaned, 'avg_rev_rentals', 'rent')
    airbnb_rev = get_avg_rev(airbnb_cleaned, 'avg_rev_airbnb', 'price')

    all_avg_rev = rentals_rev.join(airbnb_rev, on = 'zipcode', how = 'fullouter')
    all_avg_rev.show()


24/11/04 19:29:43 WARN Utils: Your hostname, PC05775 resolves to a loopback address: 127.0.1.1; using 172.21.36.156 instead (on interface eth0)
24/11/04 19:29:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/04 19:29:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/jiayin/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:130: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


DriverError: ../../data/geo/post_codes.geojson: No such file or directory

In [14]:
rentals_cleaned.where(rentals_cleaned.zipcode == '1016VV').show()

+--------------------+---------+---------------+-------+-----------+------------+-------+-----------------------+---------------------+---------+-------------+------+------+--------+----------------+--------+------------------+
|                 _id|     city|  streetAddress|zipcode|energyLabel|propertyType|areaSqm|availability_start_date|availability_end_date|roommates|smokingInside|shower|toilet|    rent|registrationCost| deposit|additionalCostsRaw|
+--------------------+---------+---------------+-------+-----------+------------+-------+-----------------------+---------------------+---------+-------------+------+------+--------+----------------+--------+------------------+
|[5d6d914f536c18a6...|Amsterdam|Lijnbaansgracht| 1016VV|    Unknown|   Apartment|   58.0|             2019-10-01|           2020-10-01|     None|           No|   Own|   Own|1,800.00|            null|3,600.00|            200.00|
+--------------------+---------+---------------+-------+-----------+------------+-------

In [22]:
pattern = r'^[a-zA-Z0-9]+$'

# Filter out rows where zipcode matches the alphanumeric pattern
df = airbnb_cleaned.filter(~F.col('zipcode').rlike(pattern))
df.show()

+-------------+------------+-------+-----------+-----------+---------------+------------+--------+------+-------------------+
|     province|municipality|zipcode|   latitude|  longitude|      room_type|accommodates|bedrooms| price|review_scores_value|
+-------------+------------+-------+-----------+-----------+---------------+------------+--------+------+-------------------+
|Noord-Holland|   Amsterdam|   ....|52.35386684|4.920099042|Entire home/apt|           4|       2| 69.00|                 90|
|Noord-Holland|   Amsterdam|`1055tk|52.37794605|4.858864841|Entire home/apt|           2|       1| 95.00|               null|
|Noord-Holland|   Amsterdam|       |52.37699446|4.878464646|Entire home/apt|           4|       2|149.00|               null|
+-------------+------------+-------+-----------+-----------+---------------+------------+--------+------+-------------------+



In [13]:
actual_rent_col = 'rent'
test = rentals_cleaned.withColumn(actual_rent_col, regexp_replace(actual_rent_col, ",", "").cast("float"))
test = test.groupBy('zipcode').agg(F.avg('rent').alias('avg_rev_rentals'))
test.where(test.zipcode == '1016VV').show()


+-------+---------------+
|zipcode|avg_rev_rentals|
+-------+---------------+
| 1016VV|         1800.0|
+-------+---------------+



In [13]:
# airbnb_cleaned.show()
airbnb_cleaned.dropna(subset=['zipcode']).show()

+-------------+------------+-------+-----------+-----------+---------------+------------+--------+------+-------------------+
|     province|municipality|zipcode|   latitude|  longitude|      room_type|accommodates|bedrooms| price|review_scores_value|
+-------------+------------+-------+-----------+-----------+---------------+------------+--------+------+-------------------+
|Noord-Holland|   Amsterdam|   1053|52.37302064|4.868460923|Entire home/apt|           4|       2|130.00|                100|
|Noord-Holland|   Amsterdam|   1095|52.36575451|4.941419235|   Private room|           2|       1| 59.00|                100|
|Noord-Holland|   Amsterdam|   1053|52.36938767|4.866972319|Entire home/apt|           4|       1| 95.00|                 90|
|Noord-Holland|   Amsterdam|   1017|52.36190508|4.888050037|Entire home/apt|           2|       1|100.00|                100|
|Noord-Holland|   Amsterdam| 1016AM|52.37153345|4.887057291|Entire home/apt|           6|       2|250.00|             

In [5]:
# from pyspark.sql.functions import col, regexp_extract, round, regexp_replace, format_number, avg
# def get_avg_rev(df, rev_col_name, actual_rent_col):
#     df = df.withColumn(actual_rent_col, regexp_replace(actual_rent_col, ",", "").cast("float"))
#     avg_rev = df.groupBy('zipcode').agg(avg(actual_rent_col).alias(rev_col_name))
#     avg_rev = avg_rev.withColumn(rev_col_name, round(col(rev_col_name), 2))\
#         .withColumn(rev_col_name, format_number(rev_col_name,2))
#     return avg_rev
rentals_rev = get_avg_rev(rentals_cleaned, 'avg_rev_rentals', 'rent')
airbnb_rev = get_avg_rev(airbnb_cleaned, 'avg_rev_airbnb', 'price')

all_avg_rev = rentals_rev.join(airbnb_rev, on = 'zipcode')
all_avg_rev.show()

+-------+---------------+--------------+
|zipcode|avg_rev_rentals|avg_rev_airbnb|
+-------+---------------+--------------+
| 1054AP|       1,972.22|        212.00|
| 1017RD|         700.00|        196.25|
| 1016AM|       1,000.00|        168.33|
| 1055ZT|         800.00|         70.33|
| 1103LH|       1,500.00|         44.00|
| 1017AA|         875.00|        217.00|
| 1051GT|       1,300.00|        190.00|
| 1013BX|       1,750.00|        136.33|
| 1095ER|       1,300.00|         85.00|
| 1019HD|       1,046.43|         99.00|
| 1094EG|       1,787.50|         89.00|
| 1016HG|       1,975.00|        183.50|
| 1072PJ|         912.50|        150.00|
| 1015JM|         900.00|        260.00|
| 1013HE|         900.00|         66.67|
| 1052JL|       1,750.00|         73.00|
| 1013HA|       1,750.00|        181.00|
| 1052LL|       1,800.00|        103.00|
| 1056XE|       1,428.33|         92.50|
| 1073GG|       1,056.67|        200.00|
+-------+---------------+--------------+
only showing top

In [26]:
airbnb_rev.show()

+-------+--------------+
|zipcode|avg_rev_airbnb|
+-------+--------------+
| 1054AP|        212.00|
| 1017RD|        196.25|
| 1013WJ|        129.00|
| 1011TW|        125.00|
| 1019HN|        120.00|
| 1016AM|        168.33|
| 1058EP|        130.00|
| 1055ZT|         70.33|
| 1103LH|         44.00|
| 1015DC|        168.00|
| 1012BN|        129.00|
| 1016XR|        135.00|
| 1017AA|        217.00|
| 1055TE|        104.50|
| 1015GN|        125.83|
| 1012da|        160.00|
|   1093|        181.69|
| 1011KW|        119.00|
| 1058TP|         75.00|
| 1071ER|        322.00|
+-------+--------------+
only showing top 20 rows



In [16]:
airbnb_rev.show()

+-------+------------------+
|zipcode|        avg(price)|
+-------+------------------+
| 1054AP|             212.0|
| 1017RD|            196.25|
| 1013WJ|             129.0|
| 1011TW|             125.0|
| 1019HN|             120.0|
| 1016AM|168.33333333333334|
| 1058EP|             130.0|
| 1055ZT| 70.33333333333333|
| 1103LH|              44.0|
| 1015DC|             168.0|
| 1012BN|             129.0|
| 1016XR|             135.0|
| 1017AA|             217.0|
| 1055TE|             104.5|
| 1015GN|125.83333333333333|
| 1012da|             160.0|
|   1093|          181.6875|
| 1011KW|             119.0|
| 1058TP|              75.0|
| 1071ER|             322.0|
+-------+------------------+
only showing top 20 rows



In [4]:
rentals_cleaned.show()
# airbnb_cleaned.show()
# airbnb.select("review_scores_value").distinct().orderBy('review_scores_value').show()

+--------------------+-------------------+--------------------+-----------+------------+-------+-----------------------+---------------------+---------+-------------+------+------+--------+----------------+--------+------------------+
|                 _id|               city|       streetAddress|energyLabel|propertyType|areaSqm|availability_start_date|availability_end_date|roommates|smokingInside|shower|toilet|    rent|registrationCost| deposit|additionalCostsRaw|
+--------------------+-------------------+--------------------+-----------+------------+-------+-----------------------+---------------------+---------+-------------+------+------+--------+----------------+--------+------------------+
|[5d2b113a43cbfd7c...|          Rotterdam|West-Varkenoordseweg|    Unknown|        Room|   14.0|             2019-06-26|                 null|        5|           No|Shared|Shared|  500.00|            0.00|  500.00|             50.00|
|[5d2b113a43cbfd7c...|          Amsterdam|         Parelstra

In [ ]:
if __name__ == "__main__":
    ingest_data()
    spark = create_spark_session()
    
    airbnb_df = get_df('../../data/airbnb.csv', spark)
    rentals_df = get_df('../../data/rentals.json', spark)
    
    rentals_cleaned= get_cleaned_df(rentals_df, 'rent', 'postalCode')
    airbnb_cleaned= get_cleaned_df(airbnb_df, 'price', 'zipcodeEnhanced')
    
    rentals_rent = get_avg_rent(rentals_cleaned, 'avg_rent_rentals')
    airbnb_rent = get_avg_rent(airbnb_cleaned, 'avg_rent_airbnb')
    
    all_avg_rent = rentals_rent.join(airbnb_rent, on = 'zipcode')
    all_avg_rent.show()
    # spark.stop()

24/11/01 16:31:37 WARN Utils: Your hostname, PC05775 resolves to a loopback address: 127.0.1.1; using 172.21.36.156 instead (on interface eth0)
24/11/01 16:31:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/01 16:31:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/jiayin/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:130: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


+-------+----------------+---------------+
|zipcode|avg_rent_rentals|avg_rent_airbnb|
+-------+----------------+---------------+
| 1054AP|        1,972.22|         212.00|
| 1017RD|          700.00|         196.25|
| 1016AM|        1,000.00|         168.33|
| 1055ZT|          800.00|          70.33|
| 1103LH|        1,500.00|          44.00|
| 1017AA|          875.00|         217.00|
| 1051GT|        1,300.00|         190.00|
| 1013BX|        1,750.00|         136.33|
| 1095ER|        1,300.00|          85.00|
| 1019HD|        1,046.43|          99.00|
| 1094EG|        1,787.50|          89.00|
| 1016HG|        1,975.00|         183.50|
| 1072PJ|          912.50|         150.00|
| 1015JM|          900.00|         260.00|
| 1013HE|          900.00|          66.67|
| 1052JL|        1,750.00|          73.00|
| 1013HA|        1,750.00|         181.00|
| 1052LL|        1,800.00|         103.00|
| 1056XE|        1,428.33|          92.50|
| 1073GG|        1,056.67|         200.00|
+-------+--

In [63]:
all_avg_rent.show()

+-------+----------------+---------------+
|zipcode|avg_rent_rentals|avg_rent_airbnb|
+-------+----------------+---------------+
| 1016VV|        1,800.00|         150.00|
| 1017NL|        1,500.00|         117.25|
| 1069TD|        1,486.25|          50.00|
| 1018GD|        1,850.00|          80.00|
| 1011EJ|          825.00|         155.00|
| 1074JA|        1,950.00|         225.00|
| 1054AP|        1,972.22|         212.00|
| 1097AH|          568.00|         160.00|
| 1078TZ|          570.00|          95.00|
| 1057JS|          250.00|         130.00|
| 1073VZ|        1,056.25|         230.00|
| 1094HW|        2,000.00|         100.00|
| 1052HV|          700.00|          82.00|
| 1017RD|          700.00|         196.25|
| 1071MH|        1,950.00|         107.00|
| 1065TR|        1,416.67|          60.00|
| 1076CW|          700.00|         136.00|
| 1016AM|        1,000.00|         168.33|
| 1055NH|          575.00|          65.00|
| 1058HD|          855.00|          99.00|
+-------+--